In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import efficientnet_b0


In [2]:
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]

train_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

val_tf = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])


In [3]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

train_ds = ImageFolder("dataset/train", transform=train_tf)
val_ds   = ImageFolder("dataset/valid", transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)


In [4]:
model = efficientnet_b0(weights="IMAGENET1K_V1")
num_classes = len(train_ds.classes)
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features,
    num_classes
)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /home/timur/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth


100.0%


In [5]:
for param in model.features.parameters():
    param.requires_grad = False

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.classifier.parameters(),
    lr=1e-3
)

cuda


In [7]:
for epoch in range(10):
    # ---------- TRAIN ----------
    model.train()
    train_loss = 0

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            loss = criterion(logits, y)

            val_loss += loss.item()
            preds = logits.argmax(1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    print(
        f"Epoch {epoch+1} | "
        f"train_loss={train_loss/len(train_loader):.4f} | "
        f"val_loss={val_loss/len(val_loader):.4f} | "
        f"val_acc={correct/total:.3f}"
    )

Epoch 1 | train_loss=0.4460 | val_loss=0.1536 | val_acc=0.957
Epoch 2 | train_loss=0.1723 | val_loss=0.1053 | val_acc=0.969
Epoch 3 | train_loss=0.1420 | val_loss=0.0878 | val_acc=0.973
Epoch 4 | train_loss=0.1268 | val_loss=0.0861 | val_acc=0.973
Epoch 5 | train_loss=0.1198 | val_loss=0.0788 | val_acc=0.975
Epoch 6 | train_loss=0.1161 | val_loss=0.0767 | val_acc=0.975
Epoch 7 | train_loss=0.1117 | val_loss=0.0754 | val_acc=0.975
Epoch 8 | train_loss=0.1106 | val_loss=0.0715 | val_acc=0.977
Epoch 9 | train_loss=0.1072 | val_loss=0.0714 | val_acc=0.977
Epoch 10 | train_loss=0.1040 | val_loss=0.0673 | val_acc=0.978


In [8]:
for param in model.features.parameters():
    param.requires_grad = True
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4
)

In [9]:
torch.save(model, "efficientnet_leaf.pkl")
torch.save(model, "efficientnet_leaf.pth")